#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [69]:
import pandas as pd
import pymongo

In [70]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [71]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [72]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [73]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

D'après le warning qu'on a eu il faut changer le type de donnée des colonnes mentionnées : ['goal', 'pledged', 'backers', 'usd pledged']

In [74]:
df_ks['goal'] = pd.to_numeric(df_ks['goal'], errors='coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors='coerce')
df_ks['backers'] = pd.to_numeric(df_ks['backers'], errors='coerce')
df_ks['usd pledged'] = pd.to_numeric(df_ks['usd pledged'], errors='coerce')
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers             float64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

In [75]:
df_ks.shape

(150000, 14)

Ensuite, je vais éliminer les valeurs 'Na' et les duplicates dans le dataframe

In [76]:
df_ks = df_ks.dropna()
df_ks = df_ks.drop_duplicates()
df_ks.shape

(148515, 14)

### Importer les données

In [77]:
collection.delete_many({})

In [78]:
collection.insert_many(df_ks.to_dict(orient='records'))

## Question 1  

Je commencerai par ordonner la collection par ordre décroissant en fonction des promesses dons 

In [79]:
collection_ordered = collection.find({}).sort("pledged", -1)

Après je prendrai les 5 premiers éléments de la collection ordonnée

In [80]:
top_dons= collection_ordered[:5]
print("Les 5 projets ayant reçu le plus de promesse de dons : ")
list(top_dons)

Les 5 projets ayant reçu le plus de promesse de dons : 


[{'_id': ObjectId('61f055afc7362c9224fd2f4a'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': 50000.0,
  'launched': '2014-07-08 10:14:37',
  'pledged': 13285226.36,
  'state': 'successful',
  'backers': 62642.0,
  'country': 'US',
  'usd pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('61f055afc7362c9224fd8974'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': 1000000.0,
  'launched': '2016-05-24 15:49:52',
  'pledged': 12779843.49,
  'state': 'successful',
  'backers': 66673.0,
  'country': 'US',
  'usd pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('61f055afc7362c9224fd3b62'),
  'ID': 2111201788,
  'name': 'Expect the Unex

## Question 2

In [81]:
success = collection.count_documents({"state":"successful"})
print ("le nombre de projets ayant atteint leur but:",success)


le nombre de projets ayant atteint leur but: 52998


## Question 3

In [82]:
# Compter le nombre de projets pour chaque catégorie.
                    
by_category = collection.aggregate([{"$group":{"_id":"$category","totalbycategory":{"$sum":1}}}])
list(by_category)

[{'_id': 'Comic Books', 'totalbycategory': 1032},
 {'_id': 'Public Art', 'totalbycategory': 1248},
 {'_id': 'Rock', 'totalbycategory': 2707},
 {'_id': 'Print', 'totalbycategory': 287},
 {'_id': 'Gaming Hardware', 'totalbycategory': 178},
 {'_id': 'Wearables', 'totalbycategory': 508},
 {'_id': 'Vegan', 'totalbycategory': 229},
 {'_id': 'Community Gardens', 'totalbycategory': 115},
 {'_id': 'Punk', 'totalbycategory': 127},
 {'_id': 'Food', 'totalbycategory': 4612},
 {'_id': 'Ceramics', 'totalbycategory': 128},
 {'_id': 'Quilts', 'totalbycategory': 32},
 {'_id': 'Comedy', 'totalbycategory': 923},
 {'_id': 'Textiles', 'totalbycategory': 105},
 {'_id': 'Dance', 'totalbycategory': 900},
 {'_id': 'Zines', 'totalbycategory': 144},
 {'_id': 'Photobooks', 'totalbycategory': 608},
 {'_id': 'Television', 'totalbycategory': 401},
 {'_id': 'Games', 'totalbycategory': 1331},
 {'_id': 'Makerspaces', 'totalbycategory': 91},
 {'_id': 'Radio & Podcasts', 'totalbycategory': 349},
 {'_id': 'Publishing', 't

In [84]:
by_main_category = collection.aggregate([{"$group":{"_id":"$main_category","totalbymaincategory":{"$sum":1}}}])
list(by_main_category)

[{'_id': 'Music', 'totalbymaincategory': 19657},
 {'_id': 'Dance', 'totalbymaincategory': 1488},
 {'_id': 'Fashion', 'totalbymaincategory': 8943},
 {'_id': 'Art', 'totalbymaincategory': 11260},
 {'_id': 'Crafts', 'totalbymaincategory': 3463},
 {'_id': 'Film & Video', 'totalbymaincategory': 24982},
 {'_id': 'Food', 'totalbymaincategory': 9805},
 {'_id': 'Design', 'totalbymaincategory': 11911},
 {'_id': 'Technology', 'totalbymaincategory': 12871},
 {'_id': 'Photography', 'totalbymaincategory': 4176},
 {'_id': 'Journalism', 'totalbymaincategory': 1807},
 {'_id': 'Theater', 'totalbymaincategory': 4305},
 {'_id': 'Comics', 'totalbymaincategory': 4139},
 {'_id': 'Games', 'totalbymaincategory': 13987},
 {'_id': 'Publishing', 'totalbymaincategory': 15721}]

## Question 4

In [95]:
# Compter le nombre de projets français ayant été instanciés avant 2016

Fr_2016 = collection.count_documents({'country': 'FR', 'launched' : {'$lt': '2016-00-00 00:00:00'}})
print("le nombre de projets français ayant été instanciés avant 2016 est : ",Fr_2016)

le nombre de projets français ayant été instanciés avant 2016 est :  330


## Question 5

In [86]:
#Récupérer les projets américains ayant demandé plus de 200 000 dollars

more_200K = collection.find({'country' : 'US', 'usd_pledged_real' :{'$gt' : 200000}})
print("Les projets américains ayant demandé plus de 200 000 dollars sont :")
list(more_200K)

Les projets américains ayant demandé plus de 200 000 dollars sont :


[{'_id': ObjectId('61f055afc7362c9224fc075c'),
  'ID': 217543389,
  'name': 'The uKeg Pressurized Growler for Fresh Beer',
  'category': 'Drinks',
  'main_category': 'Food',
  'currency': 'USD',
  'deadline': '2014-12-08',
  'goal': 75000.0,
  'launched': '2014-10-15 06:34:48',
  'pledged': 1559525.68,
  'state': 'successful',
  'backers': 10293.0,
  'country': 'US',
  'usd pledged': 1559525.68,
  'usd_pledged_real': 1559525.68},
 {'_id': ObjectId('61f055afc7362c9224fc07eb'),
  'ID': 909248984,
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2015-08-20',
  'goal': 30000.0,
  'launched': '2015-07-21 19:01:41',
  'pledged': 692912.0,
  'state': 'successful',
  'backers': 2200.0,
  'country': 'US',
  'usd pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('61f055afc7362c9224fc0898'),
  'ID': 1688905333,
  'name': 'Legion Solar - A Better Way to Energy Inde

## Question 6 

In [94]:
collection.create_index([("name","text")])
sport_inname= collection.count_documents( { "$text": { "$search": "Sport" } } )
print("Le nombre de projet ayant 'Sport' dans leur nom : ", sport_inname)

Le nombre de projet ayant 'Sport' dans leur nom :  316
